<a href="https://colab.research.google.com/github/VillePajala/Vehicle-classifier/blob/main/vehicle_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [3]:
key = os.environ.get('AZURE_SEARCH_KEY', 'xxx')

In [ ]:
results = search_images_bing(key, 'car')
ims = results.attrgot('contentUrl')
len(ims)

In [5]:
vehicle_types = 'car','bicycle','motorcycle'
path = Path('vehicles')

In [6]:
if not path.exists():
    path.mkdir()
    for o in vehicle_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o}')
        download_images(dest, urls=results.attrgot('contentUrl'))

In [ ]:
fns = get_image_files(path)
fns

In [ ]:
failed = verify_images(fns)
failed

In [9]:
failed.map(Path.unlink);

In [10]:
vehicles = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [11]:
dls = vehicles.dataloaders(path)

In [ ]:
vehicles = vehicles.new(item_tfms=Resize(128, ResizeMethod.Squish))
dls = vehicles.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

In [13]:
vehicles = vehicles.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = vehicles.dataloaders(path)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1)

In [ ]:
#hide_output
cleaner = ImageClassifierCleaner(learn)
cleaner

In [18]:
#hide
for idx in cleaner.delete(): cleaner.fns[idx].unlink()
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/bicycle)

In [19]:
learn.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

In [24]:
learn_inf = load_learner(path/'export.pkl')

In [ ]:
learn_inf.dls.vocab

In [ ]:
pred,pred_idx,probs = learn_inf.predict(img)